In [ ]:
import numpy as np
from airo_drake import DualArmScene, add_floor, add_manipulator, add_meshcat, add_wall, finish_build
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.tree import ModelInstanceIndex
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.scenes import add_cloth_competition_dual_ur5e_scene, X_W_LCB_DEFAULT, X_W_RCB_DEFAULT
from airo_drake import X_URBASE_ROSBASE


robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)

X_W_LCB = X_W_LCB_DEFAULT
X_W_RCB = X_W_RCB_DEFAULT

(arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index) = add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder, X_W_LCB, X_W_RCB
)
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

scene = DualArmScene(robot_diagram, arm_left_index, arm_right_index, gripper_left_index, gripper_right_index, meshcat)

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[
        scene.arm_left_index,
        scene.arm_right_index,
        scene.gripper_left_index,
        scene.gripper_right_index,
    ],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
from functools import partial
from cloth_tools.kinematics.constants import TCP_TRANSFORM
from cloth_tools.kinematics.inverse_kinematics import inverse_kinematics_in_world_fn

inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB, tcp_transform=TCP_TRANSFORM)
inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB, tcp_transform=TCP_TRANSFORM)

In [ ]:
from airo_planner import DualArmOmplPlanner

joint_bounds_lower = np.deg2rad([-360, -195, -160, -360, -360, -360])
joint_bounds_upper = np.deg2rad([360, 15, 160, 360, 360, 360])
joint_bounds = joint_bounds_lower, joint_bounds_upper

planner = DualArmOmplPlanner(
    is_state_valid_fn=collision_checker.CheckConfigCollisionFree,
    inverse_kinematics_left_fn=inverse_kinematics_left_fn,
    inverse_kinematics_right_fn=inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds,
)

In [ ]:
tangled_joints_left = np.deg2rad([0, -90, -90, -90, 90, 0])
tangled_joints_right = np.deg2rad([-136, -116, -110, -133, 40, 0])
tangled_joints = np.concatenate([tangled_joints_left, tangled_joints_right])

plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, tangled_joints)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

In [ ]:
from airo_typing import RotationMatrixType
from airo_drake import visualize_frame
from airo_typing import HomogeneousMatrixType

def hang_in_the_air_tcp_orientation(left: bool) -> RotationMatrixType:
    gripper_forward_direction = np.array([0, -1, 0]) if left else np.array([0, 1, 0])
    Z = gripper_forward_direction / np.linalg.norm(gripper_forward_direction)
    X = np.array([0, 0, 1]) if left else np.array([0, 0, -1])
    Y = np.cross(Z, X)
    return np.column_stack([X, Y, Z])


def hang_in_the_air_tcp_pose(left: bool) -> HomogeneousMatrixType:
    position = np.array([0, 0, 0.9])  # 1 m is too close to a singularity
    gripper_orientation = hang_in_the_air_tcp_orientation(left)

    gripper_pose = np.identity(4)
    gripper_pose[0:3, 0:3] = gripper_orientation
    gripper_pose[0:3, 3] = position
    return gripper_pose

stretch_pose_left = hang_in_the_air_tcp_pose(left=True)
stretch_pose_right = hang_in_the_air_tcp_pose(left=False)

stretch_pose_left[:3, 3] += np.array([-0.4, 0.02, 0])
stretch_pose_right[:3, 3] += np.array([-0.4, -0.02, 0])

visualize_frame(scene.meshcat, "stretch_pose_left", stretch_pose_left, opacity=0.5)
visualize_frame(scene.meshcat, "stretch_pose_right", stretch_pose_right, opacity=0.5)

In [ ]:
tangled_joints_left = np.deg2rad([0, -90, -90, -90, 90, 0])
tangled_joints_right = np.deg2rad([-136, -116, -110, -133, 40, 0])
tangled_joints = np.concatenate([tangled_joints_left, tangled_joints_right])


path = planner.plan_to_tcp_pose(tangled_joints_left, tangled_joints_right, stretch_pose_left, stretch_pose_right)

In [ ]:
from airo_drake import time_parametrize_toppra, animate_dual_joint_trajectory

trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_dual_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory)